# Transform Rasters
- This notebook is responsible for applying the profile curvature and residual topography transformations to the extent dems.
- The extent DEMs are the DEMS roughly clipped to the extent, not the exact boundary, of the ridge and swale topography of a bend. 
- The extent DEMs are required to be saved in the `data/input/dem` directory as individual `.tif` files.
- This notebook will calculate the residual topography transform of the dems and save the data in the `data/output/rt` directory
- The profile curvature transform needs to be executed in QGIS. 
- Once both transforms are completed, open [ClassifyRasters.ipynb](ClassifyRasters.ipynb) to complete the ridge area classification




In [1]:
from pathlib import Path

import numpy as np
import rasterio
import matplotlib.pyplot as plt

from scrollstats.delineation import CalcResidualTopography
from globals import RASTER_DIRS, WINDOW_SIZE

# Define Inputs


## Create the list `dem_paths`
- gather and sort all of the dems into a list
- this process expects the `input/dem` folder to contain the DEMs that cover the extent of all bends of interest to the user
                                                                                           


In [ ]:
# regex-like pattern matching to filter dems; 
# leave as `regex_str = "*.tif` to match all tifs in `dem_dir`
regex_str = "sb*.tif"


# function used to sort the dems within `dem_dir`
# The function below:

# def sort_func(x):
#     return x

# will use the default filename sorting
def sort_func(x):
    return x.stem.split("_")[2]


dem_paths = sorted(RASTER_DIRS["dem"].glob(regex_str), key = sort_func)
dem_paths[:5]

## Calculate Residual Topography
- Loop over `dem_paths` and calculate the residual topography transform for each dem present
- Save all residual topography rasters to `output/rt`

In [ ]:
for dem_path in dem_paths[:3]:
    rt = CalcResidualTopography(dem_path, WINDOW_SIZE, RASTER_DIRS["rt"])
    rt_path = rt.execute()
    print(f"Calculated rt transform for {dem_path.name}")

## DEM vs Residual Topography example 

In [ ]:
dem = rasterio.open(dem_path).read(1)
dem[dem<-1e6] = np.nan

rt = rasterio.open(rt_path).read(1)

fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(20, 15))
mapper = ax1.imshow(dem)
# fig.colorbar(mapper, ax=ax1)
ax1.set_axis_off()
ax1.set_title("Original DEM")


mapper = ax2.imshow(rt)
fig.colorbar(mapper, ax=ax2)
ax2.set_axis_off()
ax2.set_title("Residual Topography");


## Instructions for QGIS implementation of profile curvature

Profile curvature is calculated using the `r.param.scale` tool in QGIS3

The script [profileCurvature_QGIS.py](scrollstats/delineation/profileCurvature_QGIS.py) should be used to calculate the profile curvature transform on the batch of DEMs. 

Execute [profileCurvature_QGIS.py](scrollstats/delineation/profileCurvature_QGIS.py) in the python console of QGIS. You can open the python console by clicking the python symbol in the tool bar, then clicking the "show editor" button that looks like a document and pencil.

In [profileCurvature_QGIS.py](scrollstats/delineation/profileCurvature_QGIS.py), set the `dem_dir` and `output_dir` variables to the correct directories and set `window_size` to match the `WINDOW_SIZE` variable set in [globals.py](globals.py).

Additionally, set the `regex_str` and `sort_func` to match the variables of the same name in this notebook.